# ブラックボックス最適化 ハンズオン
本資料では、Optunaを使ったブラックボックス最適化を実際に体験していただきます。

# 事前準備 (乱数seedの固定)
Optunaによるブラックボックス最適化は処理の中で乱数が使われるため、通常は実行するたびに結果が変わります。

しかし今回は講義での解説用に、乱数seedを固定して実行結果が毎回同じになるようにしておきます。

以下のプログラムを最初に実行しておいてください。

In [ ]:
import numpy as np
import random

# この数値を変更すると、本資料中のプログラムの実行結果も少し変わります
SEED = 42

np.random.seed(SEED)
random.seed(SEED)

もし、これ以降のプログラムを実行したときに「変数SEEDが存在しない」というエラーとなった場合は、上記のプログラムが実行されていないことが原因です。忘れずに実行してください。

# パッケージのインストール
Google Colaboratoryは、標準ではOptunaがインストールされていません。

あらかじめ、pipコマンドでインストールします。

In [ ]:
%pip install optuna

# 簡単な数式の最適化
まずは機械学習からいったん離れて、分かりやすい簡単な数式の最適化を行ってみたいと思います。

## 今回の問題
次に示す関数$f(x, y)$の値が最小となるような$(x, y)$を見つけたいとします。

　　$f(x, y) = (1.5 - x + xy)^2 + (2.25 - x + xy^2)^2 + (2.625 - x + xy^3)^2$

　　$(-4.5 < x < 4.5, -4.5 < y < 4.5)$


数式が示されているので「ある処理」の中身は分かりますが・・・

問題の答えは簡単には分からないのではないでしょうか。これもある意味、ブラックボックスです。

【ご参考】

本数式は書籍「Optunaによるブラックボックス最適化」より引用しています。

この数式はBeale関数と言い、最適化のベンチマークに使用される代表的な「難しい関数」のひとつです。

## Optunaによる最適化プログラム
Optunaを使うと、次のようなプログラムで解を探索することができます。

まずは、細かい内容は気にせずに実行してみましょう。

In [ ]:
import optuna

# 目的関数
def objective(trialx):
    x = trialx.suggest_float("x", -4.5, 4.5)
    y = trialx.suggest_float("y", -4.5, 4.5)

    return (1.5 - x + x * y) ** 2 + \
        (2.25 - x + x * y ** 2) ** 2 + \
        (2.625 - x + x * y ** 3) ** 2

# スタディの作成
study = optuna.create_study(
    direction="minimize",
    sampler=optuna.samplers.TPESampler(seed=SEED)  # 乱数固定のため明示
    )

# スタディの実行
study.optimize(objective, n_trials=100)

# 結果の表示
print(f"Best objective value: {study.best_value}")
print(f"Best parameter: {study.best_params}")

ログの最後に出力された「Best parameter」が求めた解です。

この問題の最適解は、$(x, y) = (3, 0.5)$ですので、かなり近い値を求めることができました。

プログラム全体で15行程度しかありませんし、なんだか簡単そうだと思いませんか！？

## トライアルとスタディ
プログラムの詳細を解説する前に、Optunaによるブラックボックス最適化における重要な概念である、トライアルとスタディについて説明します。

- トライアル
  - ブラックボックス最適化では、パラメータ値を変化させながら目的関数を繰り返し評価して解を探索します。このプロセスにおける1回の評価を<font color="red">**トライアル(Trial)**</font>と呼びます。

- スタディ
  - トライアルを繰り返す一連の最適化プロセスを、<font color="red">**スタディ(Study)**</font>と呼びます。


<img src="https://github.com/tendo-sms/python_intermediate/raw/main/2024/images/trial_study.png" width="80%" alt="パレートフロントのイメージ">


これらを念頭に置いて、プログラムの内容を見ていきましょう。

## プログラムの解説
プログラムの内容について解説します。次のようなプログラムでしたね。

~~~python
import optuna

# 目的関数
def objective(trialx):
    x = trialx.suggest_float("x", -4.5, 4.5)
    y = trialx.suggest_float("y", -4.5, 4.5)

    return (1.5 - x + x * y) ** 2 + \
        (2.25 - x + x * y ** 2) ** 2 + \
        (2.625 - x + x * y ** 3) ** 2

# スタディの作成
study = optuna.create_study(
    direction="minimize",
    sampler=optuna.samplers.TPESampler(seed=SEED)  # 乱数固定のため明示
    )

# スタディの実行
study.optimize(objective, n_trials=100)

# 結果の表示
print(f"Best objective value: {study.best_value}")
print(f"Best parameter: {study.best_params}")
~~~

### Optunaパッケージのimport
import文でOptunaのパッケージを読み込みます。

~~~python
import optuna
~~~


### 目的関数の定義
次のコードで、今回の目的関数を定義しています。

~~~python
def objective(trial):
    x = trial.suggest_float("x", -4.5, 4.5)
    y = trial.suggest_float("y", -4.5, 4.5)

    return (1.5 - x + x * y) ** 2 + \
        (2.25 - x + x * y ** 2) ** 2 + \
        (2.625 - x + x * y ** 3) ** 2
~~~

Optunaにおいて、目的関数は次のルールで記述します。

- 引数をTrialオブジェクトとする。
  - `trial`という名称で引数を受け取っています。

- 戻り値を評価値とする。
  - `return`文で、戻り値として今回の$f(x, y)$の内容が記述されています。

### サジェストAPI
目的関数内で2回呼び出されている`suggest_float`メソッドは、**<font color="red">サジェストAPI</font>**と呼ばれるものです。

~~~python
    x = trial.suggest_float("x", -4.5, 4.5)
    y = trial.suggest_float("y", -4.5, 4.5)
~~~

サジェストAPIは、「今回のトライアルではどのパラメータ値を評価するのか」をOptunaが自動的に決めてくれます。

今回の例では、xとyのいずれも-4.5～4.5の浮動小数点数と設定しています。

トライアルごとに、Optunaがこの範囲内で今回のトライアルで試すべきパラメータを提案してくれます。

パラメータ名"x"は"y"は、ログの出力や、結果の可視化で表示するラベルです。


サジェストAPIは他にもありますが、それらは後ほどご紹介します。

### スタディの作成
次のコードで、Studyオブジェクトを作成しています。

~~~python
study = optuna.create_study(
    direction="minimize",
    sampler=optuna.samplers.TPESampler(seed=SEED)  # 乱数固定のため明示
    )
~~~

Studyオブジェクトの作成には、`create_study`関数を使用します。

- 引数の`direction="minimize"`は、今回の最適化が評価値の最小化を目的とすることを示します。
  - 評価値を最大化するような事例では、`direction="maximize"`と指定します。

- 引数`sampler`では、最適化のアルゴリズム(Optunaではサンプラーと呼びます)として、ベイズ最適化の一つである`TPESampler`を指定しています。
  - `TPESampler`はデフォルトのサンプラーなので通常は指定を省略できますが、今回は乱数seedを固定することで毎回同じ結果とするため、明示的にサンプラーの指定を記述しています。

### スタディの実行
次のコードで、スタディを実行しています。

~~~python
study.optimize(objective, n_trials=100)
~~~

スタディの実行には、Studyオブジェクトの`optimize`メソッドを使用します。

- 引数の`objective`は、定義した目的関数です。
- 引数の`n_trials=100`は、100回のトライアルを実施することを示します。

### 結果の表示
次のコードで、最適化の結果を表示しています。

~~~python
print(f"Best objective value: {study.best_value}")
print(f"Best parameter: {study.best_params}")
~~~

- Studyオブジェクトの`best_value`変数には、求めた解(スタディで試行した全トライアルのうち、最小の評価値)が格納されています。
- Studyオブジェクトの`best_params`には、解が`best_value`となるときのパラメータ値が格納されています。

このようにして、簡単な関数のパラメータについて、ブラックボックス最適化を行うことができました。

Optunaの機能を使えば、ごく簡単なプログラムで実現できることが体感できたのではないでしょうか。

# グリッドサーチやランダムサーチとの比較
今回の事例ではOptunaのデフォルトの手法で最適化を行いました。この手法は、ベイズ最適化によるものです。

同じ事例をグリッドサーチやランダムサーチで行った場合の、解の精度や実行速度を比較してみましょう。

Optunaは、グリッドサーチやランダムサーチの機能も提供しています。

## ベイズ最適化
まずは先ほどと同じベイズ最適化のプログラムについて、以下の処理のみ変更して実行してみます。

- 試行回数を100回から300回に変更
- スタディの実行にかかる時間を計測


In [ ]:
import time
import optuna

# 目的関数
def objective(trialx):
    x = trialx.suggest_float("x", -4.5, 4.5)
    y = trialx.suggest_float("y", -4.5, 4.5)

    return (1.5 - x + x * y) ** 2 + \
        (2.25 - x + x * y ** 2) ** 2 + \
        (2.625 - x + x * y ** 3) ** 2

# スタディの作成
study = optuna.create_study(
    direction="minimize",
    sampler=optuna.samplers.TPESampler(seed=SEED)
    )

# スタディの実行
stime = time.perf_counter()             # ★最適化の開始時間を計測
study.optimize(objective, n_trials=300) # ★試行回数を300回に変更
etime = time.perf_counter()             # ★最適化の終了時間を計測

# 結果の出力
print(f"Best objective value: {study.best_value}")
print(f"Best parameter: {study.best_params}")
print(f"Time: {etime-stime:.1f} s")  # ★最適化にかかる時間も出力

また、Optunaの可視化機能を使って、最適化の履歴も確認してみましょう。

Optunaが提供する`Visualization`オブジェクトの`plot_optimization_history`メソッドで、各トライアルでの評価値と、その時点でもっとも良い評価値の履歴を出力することができます。

なお、より詳しい可視化機能の詳細は、後ほど改めてご紹介します。

次のコードを実行してください。

In [ ]:
optuna.visualization.plot_optimization_history(
    study=study
).show()

評価値の小さいトライアルを集中的に実行できていることが分かります。

なお、右上の凡例「<font color="blue">●</font> Objective Value」をクリックすると、「<font color="red">**ー**</font> Best Value」のみを表示することができます。

最も良い評価値が探索により更新されていくことが分かります。

## グリッドサーチ
Optunaでグリッドサーチを行うには、`create_study`関数で指定する`sampler`をこれまでの`TPESampler`から`GridSampler`に変更します。

~~~python
# スタディの作成
study = optuna.create_study(
    direction="minimize",
    sampler=optuna.samplers.GridSampler(search_space)
    )
~~~

グリッドサーチにおいて、パラメータ値の候補が浮動小数点である場合は、ある程度の間隔で値を区切らなければいけません。

今回は、-4.5から-4.5までを0.1間隔で区切っています。

この場合、xとyの候補の数はそれぞれ91個となりますので、トライアル数は 91 * 91 = 8281 回となります。

プログラムは以下のとおりとなりますが、非常に時間がかかりますので、興味のある方は時間があるときに試してみてください。

~~~python
import time
import optuna

# 目的関数
def objective(trial):
    x = trial.suggest_float("x", -4.5, 4.5)
    y = trial.suggest_float("y", -4.5, 4.5)

    return (1.5 - x + x * y) ** 2 + \
           (2.25 - x + x * y ** 2) ** 2 + \
           (2.625 - x + x * y ** 3) ** 2

# GridSampler用のパラメータ範囲と分割数 (0.1間隔)
STEP = 0.1
NUM = int((4.5 - (-4.5)) / STEP)
search_space = {
    "x": [-4.5 + i * STEP for i in range(NUM+1)],  # -4.5から4.5までを0.1間隔で分割
    "y": [-4.5 + i * STEP for i in range(NUM+1)],
}

# スタディの作成
study = optuna.create_study(
    direction="minimize",
    sampler=optuna.samplers.GridSampler(search_space) # ★グリッドサーチのサンプラーを指定
    )

# スタディの実行
stime = time.perf_counter()
study.optimize(objective, n_trials=len(search_space["x"]) * len(search_space["y"]))
etime = time.perf_counter()

# 結果の出力
print(f"Best objective value: {study.best_value}")
print(f"Best parameter: {study.best_params}")
print(f"Time: {etime-stime:.1f} s")
~~~

ちなみに、結果は以下のようになります。

~~~
Best objective value: 0.0
Best parameter: {'x': 3.0, 'y': 0.5}
Time: 24754.8 s
~~~

７時間弱と非常に時間はかかりますが、最適解を含むすべてのパラメータを探索していますので、確実に最適解を求めることができます。

ただし、もしこの問題の最適解が {'x': 3.24, 'y': 0.75} などであった場合はどうなるでしょうか。

今回のパラメータ値の候補にはこれらの値がありませんので、最適解が求められないことになります。

だからといって探索範囲を0.1区切りではなく0.01区切りなどにしたら、どうなるでしょうか。

トライアル数は 901 * 901 = 811801 と100倍近くになり、探索時間も膨大になります。

グリッドサーチは、パラメータの候補が離散的な値であり、その数も少ないケースでのみ、有効な手法といえるでしょう。

【ワンポイント】

もし別の最適化問題で、パラメータの数が5個、10個だったりしたら・・・大変なことになってしまいますね。(これが、次元の罠！)

## ランダムサーチ
Optunaでランダムサーチを行うには、`create_study`関数で指定する`sampler`を`
RandomSampler`に変更します。

~~~python
# スタディの作成
study = optuna.create_study(
    direction="minimize",
    sampler=optuna.samplers.RandomSampler(seed=SEED)
    )
~~~

プログラムは次のとおりであり、サンプラーの指定以外はベイズ最適化のコードと同じです。

こちらも実行してみましょう。

In [ ]:
import time
import optuna

# 目的関数
def objective(trial):
    x = trial.suggest_float("x", -4.5, 4.5)
    y = trial.suggest_float("y", -4.5, 4.5)

    return (1.5 - x + x * y) ** 2 + \
           (2.25 - x + x * y ** 2) ** 2 + \
           (2.625 - x + x * y ** 3) ** 2

# スタディの作成
study = optuna.create_study(
    direction="minimize",
    sampler=optuna.samplers.RandomSampler(seed=SEED) # ★ランダムサーチのサンプラーを指定
    )

# スタディの実行
stime = time.perf_counter()
study.optimize(objective, n_trials=300)
etime = time.perf_counter()

# 結果の出力
print(f"Best objective value: {study.best_value}")
print(f"Best parameter: {study.best_params}")
print(f"Time: {etime-stime:.1f} s")

結果はまぁまぁといったところでしょうか。

ランダムな試行なので、運にも左右されます。試行回数を多くすることで最適解に近づく確率は高まりますが、実行時間とのトレードオフとなります。

こちらも、最適化がどのように行われたかの履歴を確認してみましょう。

In [ ]:
optuna.visualization.plot_optimization_history(
    study=study
).show()

ランダムなので当然ではありますが、スコアの高くないトライアルも多数行われています。

複雑な最適化問題においては、ベイズ最適化と比べてかなり効率が悪い探索となってしまうことが推測できます。

## 最適化実行時間の比較
実行するタイミングによって結果に幅があるかとは思いますが、こちらで事前に試した結果では、以下となりました。

- グリッドサーチ：24754.8 s
- ランダムサーチ：0.9 s
- ベイズ最適化：4.9 s


傾向として、実行時間の大小が以下の関係であることが確認できました。

　　グリッドサーチ ＞＞ ベイズ最適化 ＞ ランダムサーチ

ランダムサーチとベイズ最適化の比較については、ベイズ最適化自体が計算コストのかかる処理なので、それが原因で差異が出ています。

# 機械学習におけるハイパーパラメータの最適化
次はいよいよ、機械学習のハイパーパラメータ最適化にOptunaを活用してみましょう。

今回は、機械学習の講義でも使用した`california housing dataset`を用いて、ランダムフォレスト回帰による回帰タスクを行うサンプルをご紹介します。

## データの読み込み
機械学習の講義では、欠損値を混ぜてアレンジしたデータを使用しましたが、今回は`scikit-learn`が提供するデータをそのまま使用します。

`fetch_california_housing`関数で`california housing dataset`を読み込みます。

In [ ]:
from sklearn.datasets import fetch_california_housing

# california housing datasetの読み込み
data = fetch_california_housing()
X, y = data.data, data.target

## データセットの分割と件数の絞り込み
`train_test_split`を使って、データセットを訓練データとテストデータに分割します。

今回は、全データのうち800件を訓練データ、200件をテストデータとして使用します。

次のコードを実行してください。

In [ ]:
from sklearn.model_selection import train_test_split

SIZE_TRAIN = 800
SIZE_TEST = 200

# 訓練データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=SIZE_TRAIN, test_size=SIZE_TEST, random_state=SEED
)

## ハイパーパラメータ最適化前の実行結果
`scikit-learn`では、ランダムフォレスト回帰のための`RandomForestRegressor`が提供されています。

まずは、デフォルトパラメータでの精度を確認します。指標としてはMSEを使用します。

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# デフォルトパラメータでの精度確認
rf_default = RandomForestRegressor(random_state=SEED)
rf_default.fit(X_train, y_train)
y_predict = rf_default.predict(X_test)

mse_default = mean_squared_error(y_test, y_predict)
print(f"MSE (Default): {mse_default:.4f}")

ここで算出した`mse_default`は、この後ご紹介する様々な最適化手法において、最適化前の評価値として何度も使用します。

## ハイパーパラメータ最適化のプログラム作成
ここからいよいよ、ハイパーパラメータの最適化にチャレンジしていきます。

### RandomForestRegressorのハイパーパラメータ
講義資料でご紹介したとおり、RandomForestRegressorの代表的なハイパーパラメータとしては以下などがあります。

- n_estimators：決定木の数
- max_depth：決定木の最大の深さ
- max_features：分岐を決定するのに用いる説明変数の最大数

今回はこれらのハイパーパラメータについて、Optunaを使って最適化してみましょう。

### 目的関数の作成
目的関数を設定します。

トライアルごとにOptunaから提案されたハイパーパラメータ値で`RandomForestRegressor`のインスタンスを作成し、学習した結果のMSEを戻り値とします。

次のコードを実行してください。

In [ ]:
def objective(trial):
    # 目的関数
    n_estimators = trial.suggest_int("n_estimators", 50, 300)
    max_depth = trial.suggest_int("max_depth", 3, 30)
    max_features = trial.suggest_categorical("max_features", ["sqrt", "log2"])

    rf_trial = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        max_features=max_features,
        random_state=SEED
    )

    rf_trial.fit(X_train, y_train)
    y_predict = rf_trial.predict(X_test)

    mse_trial = mean_squared_error(y_test, y_predict)
    return mse_trial

## サジェストAPIの解説
OptunaのサジェストAPIには、以下のような種類があります。

- `suggest_int`
- `suggest_float`
- `suggest_categorical`

前半では`suggest_float`を使用しましたが、今回は次のとおり、`suggest_int`と`suggest_categorical`を使用しました。

~~~
n_estimators = trial.suggest_int("n_estimators", 50, 300)
max_depth = trial.suggest_int("max_depth", 3, 30)
max_features = trial.suggest_categorical("max_features", ["sqrt", "log2"])
~~~

それぞれのサジェストAPIについて、詳しくご紹介します。

### suggest_int、suggest_float
パラメータの候補が整数型のときは`suggest_int`、浮動小数点数型のときは`suggest_float`を、それぞれ使用します。

文法は共通で、次のとおりです。

~~~python
suggest_int(name, low, high, *[, step, log])
suggest_float(name, low, high, *[, step, log])
~~~
- `name`は、パラメータに付ける名前です。ログ出力や可視化においてラベルとして用いられます。
- `low`、`high`は、パラメータ値の候補の最小値、最大値です。
- `step`が指定された場合、`step`で区切られた離散的なパラメータ値が決定されます。
- `log`が指定された場合、対数スケールでパラメータ値が決定されます。

前半では、値の範囲が$(-4.5 < x < 4.5, -4.5 < y < 4.5)$であることを示すため、`suggest_float`を用いました。

今回は、`n_estimators`と`max_depth`の候補値が整数型でしたので、`suggest_int`を用いました。

### suggest_categorical
パラメータの候補が文字列ラベルの一覧といった名義尺度であるときは、`suggest_categorical`を使用します。

文法は次のとおりです。

~~~python
suggest_categorical(name, choices)
~~~
- `name`は、パラメータに付ける名前です。
- `choices`は、パラメータ値の候補となるリストです。

今回は、`max_features`の候補値が文字列ラベルから選択するものであったため、`suggest_categorical`を用いました。

### スタディの作成および実行 ～ 結果の出力
ここからは、入門編でご紹介した使い方とまったく同じです。

次のとおり、スタディの作成、実行、結果の出力を行います。

今回の試行回数は、40回としました。

次のコードを実行してください。

In [ ]:
import optuna

# スタディの作成
study = optuna.create_study(
    direction="minimize",
    sampler=optuna.samplers.TPESampler(seed=SEED)  # 乱数固定のため明示
    )

# スタディの実行
study.optimize(objective, n_trials=40)

# 結果の出力
print(f"Best objective value: {study.best_value}")
print(f"Best parameter: {study.best_params}")
print("")

# デフォルトパラメータとの比較
print(f"デフォルト MSE: {mse_default:.4f}")
print(f"チューニング後 MSE: {study.best_value:.4f}")
print(f"改善率: {(mse_default - study.best_value)/mse_default*100:.2f}%")

デフォルトの場合と比較して、数%の向上が見られました。

ハイパーパラメータのブラックボックス最適化も、入門編でご紹介した数式の最適化と全く同じやり方で、簡単に実践できることが体感できたかと思います。

### [ご参考] 全体を1つにまとめたプログラム
ここまでで、Optunaによるハイパーパラメータ最適化のプログラムが実行できました。

説明の都合上プログラムを分割して順番に動かしてきましたが、ご参考までに、全体を1つにまとめたソースコードを掲載します。

~~~python
import numpy as np
import random
import optuna
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# この数値を変更すると、本資料中のプログラムの実行結果も少し変わります
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

SIZE_TRAIN = 800
SIZE_TEST = 200

# california housing datasetの読み込み
data = fetch_california_housing()
X, y = data.data, data.target

# 訓練データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=SIZE_TRAIN, test_size=SIZE_TEST, random_state=SEED
)

# デフォルトパラメータでの精度確認
rf_default = RandomForestRegressor(random_state=SEED)
rf_default.fit(X_train, y_train)
y_predict = rf_default.predict(X_test)

mse_default = mean_squared_error(y_test, y_predict)
print(f"MSE (Default): {mse_default:.4f}")

def objective(trial):
    # 目的関数
    n_estimators = trial.suggest_int("n_estimators", 50, 300)
    max_depth = trial.suggest_int("max_depth", 3, 30)
    max_features = trial.suggest_categorical("max_features", ["sqrt", "log2"])

    rf_trial = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        max_features=max_features,
        random_state=SEED
    )

    rf_trial.fit(X_train, y_train)
    y_predict = rf_trial.predict(X_test)
    
    mse_trial = mean_squared_error(y_test, y_predict)
    return mse_trial

# スタディの作成
study = optuna.create_study(
    direction="minimize",
    sampler=optuna.samplers.TPESampler(seed=SEED)  # 乱数固定のため明示
    )

# スタディの実行
study.optimize(objective, n_trials=40)

# 結果の出力
print(f"Best objective value: {study.best_value}")
print(f"Best parameter: {study.best_params}")
print("")

# デフォルトパラメータとの比較
print(f"デフォルト MSE: {mse_default:.4f}")
print(f"チューニング後 MSE: {study.best_value:.4f}")
print(f"改善率: {(mse_default - study.best_value)/mse_default*100:.2f}%")
~~~

## 結果の可視化と分析
Optunaでは、スタディの結果を可視化・分析するための様々な機能が提供されています。

これらの機能でパラメータと評価値の関係などを分析することで、さらに効率の良い探索を行う方法を検討できます。

ここでは、いくつかの機能をご紹介します。

### データフレームの出力
Studyオブジェクトの`trials_dataframe`メソッドで、各トライアルの結果を`pandas`のデータフレームとして出力できます。

次のコードを実行してください。

In [ ]:
df_trial = study.trials_dataframe()
df_trial


データフレームは1行が1回のトライアルとなっており、トライアルに関する次のような情報が格納されます。

- 通し番号
- 評価値
- 開始・終了時刻・経過時間
- 試行したパラメータ値
- ステータス

### ハイパーパラメータ重要度の出力
Visualizationオブジェクトの`plot_param_importances`メソッドで、それぞれのハイパーパラメータがどのくらい評価結果に影響したかが分かります。

この影響度は「**ハイパーパラメータ重要度**」と呼ばれます。

次のコードを実行してください。

In [ ]:
optuna.visualization.plot_param_importances(
    study=study,
    params=["n_estimators", "max_depth", "max_features"]
).show()

この結果から、今回のチューニングでは`max_depth`の影響が大部分を占める傾向が見られます。

一方、`n_estimators`および`max_features`の影響度は小さかったことが分かります。

影響度が非常に小さいパラメータは最適化の対象から外す、などの対応が考えられます。

### パラメータ値と評価値の関係を出力
Visualizationオブジェクトの`plot_slice`メソッドで、それぞれのハイパーパラメータと評価値の関係を散布図により詳細に可視化することができます。

次のコードを実行してください。

In [ ]:
optuna.visualization.plot_slice(
    study=study,
    params=["n_estimators", "max_depth", "max_features"]
).show()

この機能は、パラメータ値の候補を絞り込むときに大変有効です。

たとえば、値がある範囲を外れたら評価値がほとんど同じになる、などの傾向があれば、その部分は候補から外すことで無駄な探索を減らすことができます。

また、パラメータ値と評価値に明確な相関があれば、評価値が低くなる方向の候補は削り、高くなる方向の候補はもっと増やすなどすることができます。

### 等高線プロットの出力
Visualizationオブジェクトの`plot_contour`メソッドで、2つのハイパーパラメータの組み合わせと評価値の関係を、等高線図により詳細に可視化することができます。

例えば、`n_estimators`と`max_depth`の値と、評価値との関係について、等高線図を作成してみます。

次のコードを実行してください。

In [ ]:
optuna.visualization.plot_contour(
    study=study,
    params=["n_estimators", "max_depth"]
).show()

また、`max_features`も加えた3つのパラメータをプロットすると、組み合わせのパターンごとに等高線図が作成されます。

次のコードを実行してください。

In [ ]:
optuna.visualization.plot_contour(
    study=study,
    params=["n_estimators", "max_depth", "max_features"]
).show()

パラメータの組み合わせと評価値の関係を分析したいときに、活用したい機能です。

### 探索履歴の出力
前半でもご紹介した機能ですが、Visualizationオブジェクトの`plot_optimization_history`メソッドで、各トライアルでの評価値と、その時点でもっとも良い評価値の履歴を出力することができます。

次のコードを実行してください。

In [ ]:
optuna.visualization.plot_optimization_history(
    study=study
).show()

ベイズ最適化であれば、トライアルが進むごとに評価値の高い試行が増えていく傾向が見えれば、効率的な探索ができていると言えそうです。

# 事前知識の活用
ベイズ最適化において、最初の方のトライアルではパラメータ値の候補を選ぶ基準がないため、ほぼランダムでパラメータ値の候補が決定します。

そのため、的外れなパラメータも含めた大量のトライアルを重ねることで、しだいに効率的な探索が行われていきます。

しかし、もし「あるていど良い結果が出るパラメータ値」が事前に分かっていれば、探索のスタートダッシュを有利にすることができます。

## 探索の初期値設定
先ほどの最適化で、以下の解が得られました。

~~~
Best objective value: 0.36670699054153133
Best parameter: {'n_estimators': 188, 'max_depth': 14, 'max_features': 'sqrt'}
~~~

少なくともこのパラメータは良い結果が出ることが分かっているので、事前知識として活用できそうです。

また、パラメータのデフォルト値は平均的によい結果となる値が採用されているものですので、これも事前知識として活用しましょう。

`max_depth`と`max_features`のデフォルト値はちょっと特殊なので省略しますが、`n_estimators`のデフォルト値は100です。

Studyオブジェクトの`enqueue_trial`メソッドを使って、これらの値から探索を開始するよう指示することができます。

~~~python
study.enqueue_trial({"n_estimators": 188, "max_depth": 14, "max_features": "sqrt"})
study.enqueue_trial({"n_estimators": 100})
~~~
この処理を追加した、以下のプログラムを実行してみましょう。

次のコードを実行してください。

In [ ]:
# スタディの作成
study = optuna.create_study(
    direction="minimize",
    sampler=optuna.samplers.TPESampler(seed=SEED)
    )

# スタディの実行
study.enqueue_trial({"n_estimators": 188, "max_depth": 14, "max_features": "sqrt"})
study.enqueue_trial({"n_estimators": 100})
study.optimize(objective, n_trials=10)  # 試行回数は10回とする

# 結果の出力
print(f"Best objective value: {study.best_value}")
print(f"Best parameter: {study.best_params}")

今回は設定が有効になったかを確認するだけを目的として、トライアルは10回としています。

ログの最初の2行を見てみてください。

`enqueue_trial`メソッドで指定したパラメータの探索を最初に行っていることが確認できます。

ただし、今回のような小規模なサンプル、かつ少ない初期値設定では、明確な効果は出にくいです。

## 評価済みトライアルの追加
先ほどは、パラメータの値のみを事前知識として利用しました。

それだけでなく、そのパラメータを使用したときの評価値まで判明している場合は、評価値も含めて事前知識として活用できます。

これを行うには、Studyオブジェクトの`add_trial`メソッドを使用します。

~~~python
# 探索空間の設定が必要
search_space = {
    "n_estimators": optuna.distributions.IntDistribution(50, 300),
    "max_depth": optuna.distributions.IntDistribution(3, 30),
    "max_features": optuna.distributions.CategoricalDistribution(["sqrt", "log2"])
}

# トライアルの追加
study.add_trial(optuna.trial.create_trial(
    params={"n_estimators": 188, "max_depth": 14, "max_features": "sqrt"},
    distributions=search_space,
    value=0.36670699054153133
))
~~~

こちらも今回のように講義用の小規模なサンプルではあまり影響が出ないため、実行可能なサンプルコードは省略します。

より効率的な探索を追求する場合は、活用してみてください。

# 交差検証による評価
これまで、データセットを訓練データとテストデータに分け、テストデータでモデルを評価する方法をご紹介してきました。

ここでは、もっと簡単に、かつ精度の高い評価を行う手法として、**交差検証(クロスバリデーション)**をご紹介します。

## データの準備
交差検証では、データセットを訓練データとテストデータに分割する必要がありません。

すべてのデータを使うこともできますが、交差検証は時間がかかるため、今回は講義用にすべてのデータから500件のデータをランダムに取り出して使用します。

`train_test_split`を応用して取り出すことができます。

次のコードを実行してください。

In [ ]:
SIZE_DATA = 500

# ランダムにデータをサンプリング
X_cv, _, y_cv, _ = train_test_split(X, y, train_size=SIZE_DATA, random_state=SEED)

`train_size`引数で、訓練データを500件取り出し、テストデータは捨ててしまいます。

こうして得られた訓練データを、訓練データとしてではなく、交差検証用のデータとして使用します。

## 目的関数の作成
目的関数を作成します。

次のコードを実行してください。

In [ ]:
from sklearn.model_selection import cross_val_score

def objective(trial):
    # 目的関数
    n_estimators = trial.suggest_int("n_estimators", 50, 300)
    max_depth = trial.suggest_int("max_depth", 3, 30)
    max_features = trial.suggest_categorical("max_features", ["sqrt", "log2"])

    rf_trial = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        max_features=max_features,
        random_state=SEED
    )

    # 交差検証によるモデルの評価
    score = cross_val_score(rf_trial, X_cv, y_cv, cv=3, scoring="neg_mean_squared_error")
    return -1 * score.mean()

**＜説明に入る前に・・・さらに次のコードもついでに実行しておきましょう。次のコードは実行に時間がかかるので。。。＞**

モデルの定義は先ほどと同じですが、モデルの評価の部分が異なっています。

交差検証を用いる場合、データをテストデータと訓練データに分割したり、コード中で`model.fit()`を行う必要もありません。

`scikit-learn`の機能である`cross_val_score`関数が、以下のことを自動的に行っています。

1. データを3つのグループに分割します。分割数は引数`cv`で指定しています。
2. 分割した1個めをテストデータ、2個目と3個目を訓練データとして、学習(`model.fit()`)と評価を行います。
3. 分割した2個めをテストデータ、1個目と3個目を訓練データとして、学習と評価を行います。
4. 分割した3個めをテストデータ、1個目と2個目を訓練データとして、学習と評価を行います。
5. 上記2.～4.の評価結果を、リストとして返します。

評価値は、これまでと同様にMSEを使います。

最終的に、すべての評価におけるMSEの平均を評価値としています。

`cross_val_score`ではMSEとしてマイナスの値となる`neg_mean_squared_error`のみ指定できますので、-1を掛けた結果を戻り値としています。

なお評価値を何も指定しなかった場合は、採用する機械学習モデルにより自動的に決定されます。

## スタディの作成および実行 ～ 結果の出力
スタディの作成、実行、結果の出力はこれまでと同じです。

次のコードを実行してください。

In [ ]:
# スタディの作成
study = optuna.create_study(
    direction="minimize",
    sampler=optuna.samplers.TPESampler(seed=SEED)
    )

# スタディの実行
study.optimize(objective, n_trials=40)

# 結果の出力
print(f"Best objective value: {study.best_value}")
print(f"Best parameter: {study.best_params}")
print("")

# デフォルトパラメータとのMSE比較 (最初の最適化と同じ訓練データおよびテストデータを使用)
best_model = RandomForestRegressor(**study.best_params, random_state=SEED)
best_model.fit(X_train, y_train)
y_predict_cv = best_model.predict(X_test)
mse_tuned = mean_squared_error(y_test, y_predict_cv)

print(f"デフォルト MSE: {mse_default:.4f}")
print(f"チューニング後 MSE: {mse_tuned:.4f}")
print(f"改善率: {(mse_default - mse_tuned)/mse_default*100:.2f}%")

交差検証は、評価に時間がかかるのが難点です。

講義用にデータ数を少なめとしたことに加え、交差検証はより厳しいチェックとなりますので、最初の最適化の時と比べて改善率は低下しました。

とはいえコード自体は非常に簡単ですし、訓練データおよびテストデータの偏りを低減することで、より堅牢な結果が期待できます。



交差検証はハイパーパラメータの決定だけでなく、機械学習のプロセスの中で次のような用途でも非常に強力です。

- 学習アルゴリズムの選択
- 特徴量設計の妥当性評価

便利な機能なので、ぜひ覚えておきましょう。

## [ご参考] 全体を1つにまとめたプログラム
ここまでで、交差検証を利用したプログラムが実行できました。

説明の都合上プログラムを分割して順番に動かしてきましたが、ご参考までに、全体を1つにまとめたソースコードを掲載します。

~~~python
import numpy as np
import random
import optuna
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

# この数値を変更すると、本資料中のプログラムの実行結果も少し変わります
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

SIZE_TRAIN = 800
SIZE_TEST = 200
SIZE_DATA = 500

# california housing datasetの読み込み
data = fetch_california_housing()
X, y = data.data, data.target

# 訓練データとテストデータに分割 (最終評価用)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=SIZE_TRAIN, test_size=SIZE_TEST, random_state=SEED
)

# ランダムにデータをサンプリング
X_cv, _, y_cv, _ = train_test_split(X, y, train_size=SIZE_DATA, random_state=SEED)

def objective(trial):
    # 目的関数
    n_estimators = trial.suggest_int("n_estimators", 50, 300)
    max_depth = trial.suggest_int("max_depth", 3, 30)
    max_features = trial.suggest_categorical("max_features", ["sqrt", "log2"])

    rf_trial = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        max_features=max_features,
        random_state=SEED
    )

    # 交差検証によるモデルの評価
    score = cross_val_score(rf_trial, X_cv, y_cv, cv=3, scoring="neg_mean_squared_error")
    return -1 * score.mean()

# スタディの作成
study = optuna.create_study(
    direction="minimize",
    sampler=optuna.samplers.TPESampler(seed=SEED)
    )

# スタディの実行
study.optimize(objective, n_trials=40)

# 結果の出力
print(f"Best objective value: {study.best_value}")
print(f"Best parameter: {study.best_params}")
print("")

# デフォルトパラメータとのMSE比較 (最初の最適化と同じ訓練データおよびテストデータを使用)
default_model = RandomForestRegressor(random_state=SEED)
default_model.fit(X_train, y_train)
y_predict = default_model.predict(X_test)
mse_default = mean_squared_error(y_test, y_predict)

best_model = RandomForestRegressor(**study.best_params, random_state=SEED)
best_model.fit(X_train, y_train)
y_predict_cv = best_model.predict(X_test)
mse_tuned = mean_squared_error(y_test, y_predict_cv)

print(f"デフォルト MSE: {mse_default:.4f}")
print(f"チューニング後 MSE: {mse_tuned:.4f}")
print(f"改善率: {(mse_default - mse_tuned)/mse_default*100:.2f}%")
~~~

# 多目的最適化
これまでご紹介した例は、評価値が1つでした。しかし現実的には、評価値が2つ以上になるような事例も考えられます。

身近な例で例えると、スマートフォンを購入しようと思ったとき、「価格」「性能」という2つを評価値とすることが考えられます。

これらはどちらか一方が優れているだけではダメで、「あちら立てればこちらが立たぬ」というトレードオフの関係となっています。

機械学習のハイパーパラメータ最適化においては、例えば「精度」に加えて、学習や評価にかかる「時間」の2つを評価値とするパターンが考えられます。

これも一般的には「あちら立てればこちらが立たぬ」のパターンです。

このように、複数の評価値を最大化・最小化するための最適化を、多目的最適化と呼びます。

ここでは、最初の決定係数による評価を行う最適化プログラムをベースに、「評価値の精度」と「最適化の時間」の2つを最適化する例をご紹介します。

## 目的関数の作成
目的関数を作成します。

次のコードを実行してください。

In [ ]:
import time

def objective(trial):
    # 目的関数
    n_estimators = trial.suggest_int("n_estimators", 50, 300)
    max_depth = trial.suggest_int("max_depth", 3, 30)
    max_features = trial.suggest_categorical("max_features", ["sqrt", "log2"])

    rf_trial = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        max_features=max_features,
        random_state=SEED
    )

    start_time = time.time() # 開始時間

    rf_trial.fit(X_train, y_train)
    y_predict = rf_trial.predict(X_test)
    mse_trial = mean_squared_error(y_test, y_predict)

    end_time = time.time()   # 終了時間

    return mse_trial, (end_time - start_time)

モデルの定義は先ほどと同じですが、次の点が異なります。

- 最適化にかかる時間として、学習と評価の合計時間を測定
- 戻り値に評価値と測定時間のタプルを指定

## スタディの作成
スタディを作成します。

次のコードを実行してください。

In [ ]:
# スタディの作成
study = optuna.create_study(
    directions=["minimize", "minimize"],
    sampler=optuna.samplers.NSGAIISampler(seed=SEED) # 乱数固定のため明示
    )

作成にはこれまでのとおり`create_study`を使用しますが、引数が異なります。

まず、これまでの`direction`に代わり、`directions`を指定しています。

~~~python
directions=["minimize", "minimize"],
~~~

`directions`の引数はリストであり、1つ目の評価値(MSE)を最小に、2つ目の評価値(時間)も最小となるよう、`["minimize", "minimize"]`としています。

また、`sampler`にはこれまでのTPESamplerではなく、NSGAIISamplerを指定しています。

~~~python
sampler=optuna.samplers.NSGAIISampler(seed=SEED)  # 乱数固定のため明示
~~~

NSGAIISamplerは、多目的最適化においてTPESamplerよりも高いパフォーマンスを発揮します。

NSGAIISamplerはベイズ最適化ではなく、進化計算と呼ばれるアルゴリズムをベースとしたサンプラーです。

なお`sampler`の指定を省略した場合、単目的最適化であればTPESamplerを、多目的最適化であればNSGAIISamplerが、自動的にデフォルトとして使用されます。

## [ご参考] Optunaが提供するサンプラー
Optunaは、次のようなサンプラーを提供しています。

- BaseSampler
- GridSampler
- RandomSampler
- TPESampler (単目的最適化のデフォルト)
- CmaEsSampler
- GPSampler
- PartialFixedSampler
- NSGAIISampler (多目的最適化のデフォルト)
- NSGAIIISampler
- QMCSampler
- BruteForceSampler

それぞれの特徴が公式リファレンスにまとめられていますので、興味のある方は見てみてください。
([optuna.samplers](https://optuna.readthedocs.io/en/stable/reference/samplers/))

また、Optuna向け機能共有プラットフォーム「OptunaHub」にて、AutoSamplerという機能が公開されています。

これを利用すると、試行回数、パラメータ数、探索空間などから最適と見込まれるサンプラーが自動的に選択されます。(optunahubパッケージのインストールが必要です。)

~~~
study = optuna.create_study(
    sampler=optunahub.load_module(
        "samplers/auto_sampler"
    ).AutoSampler()  # Automatically selects an algorithm internally
)
~~~
参考記事：[AutoSampler: Automatic Selection of Optimization Algorithms in Optuna](https://medium.com/optuna/autosampler-automatic-selection-of-optimization-algorithms-in-optuna-1443875fd8f9)

## スタディの実行と結果の出力
スタディを実行し、結果を出力します。

次のコードを実行してください。

In [ ]:
# スタディの実行
study.optimize(objective, n_trials=40)

# 結果の出力
print("[Best Trials]")
for trial in study.best_trials:
    print(f"- [{trial.number}] params={trial.params}, values={trial.values}")

スタディの実行はこれまでと同じです。試行回数は40としています。

今回のような多目的最適化では、前述のとおり評価結果が「あちら立てればこちらが立たぬ」の状態です。

そのため、最適なパラメータの組み合わせを一つに確定することはできません。

スタディの結果は、`best_trials`という変数にリストとして格納されています。これを最適化の結果として出力します。

~~~python
print("[Best Trials]")
for trial in study.best_trials:
    print(f"- [{trial.number}] params={trial.params}, values={trial.values}")
~~~

実行結果を見ると、複数の解が出力されていることが分かります。

このリストに含まれる結果は「**パレートフロント**」と呼ばれており、「他のすべてのトライアルに対して、いずれかの評価値が優れている」結果の集合です。


## [参考] パレートフロントのイメージ
パレートフロントのイメージを図示します。このイメージでは、評価1、評価2の両方とも、値が小さい方が優れているとします。

<img src="https://github.com/tendo-sms/python_intermediate/raw/main/2024/images/pareto_front.png" width="35%" alt="パレートフロントのイメージ">

この例でのパレートフロントは、①と②になります。

- ①は、評価1において他の全てのトライアルに勝っています。
- ②は、評価2において他の全てのトライアルに勝っています。
- ③と④は、評価1と評価2のどちらも、他のいずれかのトライアルに劣っています。

## [ご参考] 全体を1つにまとめたプログラム
ここまでで、Optunaによる多目的最適化のプログラムが実行できました。

説明の都合上プログラムを分割して順番に動かしてきましたが、ご参考までに、全体を1つにまとめたソースコードを掲載します。

このコードでは、乱数seedを固定していません。そのため、`create_study`関数で`sampler`を指定する必要はありません。

~~~python
import numpy as np
import random
import optuna
import time
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# この数値を変更すると、本資料中のプログラムの実行結果も少し変わります
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

SIZE_TRAIN = 800
SIZE_TEST = 200

# california housing datasetの読み込み
data = fetch_california_housing()
X, y = data.data, data.target

# 訓練データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=SIZE_TRAIN, test_size=SIZE_TEST, random_state=SEED
)

def objective(trial):
    # 目的関数
    n_estimators = trial.suggest_int("n_estimators", 50, 300)
    max_depth = trial.suggest_int("max_depth", 3, 30)
    max_features = trial.suggest_categorical("max_features", ["sqrt", "log2"])

    rf_trial = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        max_features=max_features,
        random_state=SEED
    )

    start_time = time.time() # 開始時間

    rf_trial.fit(X_train, y_train)
    y_predict = rf_trial.predict(X_test)
    mse_trial = mean_squared_error(y_test, y_predict)

    end_time = time.time()   # 終了時間

    return mse_trial, (end_time - start_time)

# スタディの作成
study = optuna.create_study(
    directions=["minimize", "minimize"],
    sampler=optuna.samplers.NSGAIISampler(seed=SEED)  # 乱数固定のため明示
    )

# スタディの実行
study.optimize(objective, n_trials=40)

# 結果の出力
print("[Best Trials]")
for trial in study.best_trials:
    print(f"- [{trial.number}] params={trial.params}, values={trial.values}")
~~~

## 評価結果の可視化
結果のリストをただ並べても、その中でどれを選ぶべきか、ぱっと見では判断できないでしょう。

そこで、評価結果を可視化してみます。Visualizationオブジェクトの`plot_pareto_front`メソッドで、トライアルの結果とパレートフロントを可視化できます。

次のコードを実行してください。

In [ ]:
# すべてのトライアルをプロット
optuna.visualization.plot_pareto_front(
	   study,
	   target_names=["MSE", "Time"]
	   ).show()

横軸を精度、縦軸を時間として、各トライアルの結果がプロットされています。

精度が良くなるほど速度は遅くなる傾向が分かります。

赤いカラーバーで示される点が`best_trials`(パレートフロント)、青いカラーバーで示される点がその他のトライアル結果です。

また次のとおり、`include_dominated_trials`引数にFalseを指定して`plot_pareto_front`メソッドを実行すると、パレートフロントのみプロットすることができます。

次のコードを実行してください。

In [ ]:
# Study.best_trials だけをプロット
optuna.visualization.plot_pareto_front(
	   study,
	   target_names=["MSE", "Time"],
	   include_dominated_trials=False
	   ).show()

パレートフロントのみが表示されました。

多目的最適化では、パレートフロントの結果の中から、用途や目的に合わせてどの結果を採用するのか決めるのがよいでしょう。

#パラメータ値の候補を決めるには
パラメータ値の候補数や、そもそものパラメータ数が多すぎると・・・
- たとえベイズ最適化を使っても、満足のいく結果を得られるまで時間がかかりすぎる
- 数時間どころか、数日かかるなんてことも

結局、無駄なパラメータ値候補やパラメータを減らして、探索範囲を必要最小限に減らすのが非常に有効
- だけど・・・数を減らそうにも、ブラックボックスなので、どうすれば適切か全然わからない (大いなる矛盾！！)

## パラメータ値の候補・パラメータ数を減らすヒント その1
scikit-learnのリファレンスなどでデフォルト値を調べて、そこから逸脱しない範囲をパラメータ値の候補に選ぶ
- デフォルト値に選ばれるだけあり、多くのケースで優れていることが多い (専門家の知恵)

## パラメータ値の候補・パラメータ数を減らすヒント その2
ランダムサーチである程度のトライアルを試行し、パラメータ値と評価値の関係を分析する
- 平均して評価値の高い、パラメータ値の範囲の見極め
- パラメータ値と評価値の相関が極端に低いようであれば、最適化対象から外してデフォルト値を採用 (パラメータ数自体を減らしてしまう)
- ご紹介した`plot_param_importances`メソッドや`plot_slice`メソッドを有効活用

<img src="https://github.com/tendo-sms/python_intermediate/raw/main/2025/figure/plot_param_importances.png" width="80%" alt="plot_param_importancesによる可視化">

<img src="https://github.com/tendo-sms/python_intermediate/raw/main/2025/figure/plot_slice.png" width="80%" alt="plot_sliceによる可視化">

# [ご参考] 分散並列最適化
分散並列最適化は、複数ノードを活用して、効率的にハイパーパラメータ探索を行う手法です。

- Optunaの機能を活用して簡単に実行可能
- トライアルを並列実行することで、探索スピードと精度が向上
- トライアルの結果をRDBに集約し、各ノードがアクセスすることで並列化を実現

<img src="https://github.com/tendo-sms/python_intermediate/raw/main/2025/figure/dist_para_opt.png" width="100%" alt="分散並列最適化">


# 最後に本音を少し
scikit-learnはハイパーパラメータのデフォルト値がかなり優秀で、特に前処理をしっかり行ったデータでは、最適化の効果はそれほど大きくないことも多いです。

(今回のサンプルコードに関しても、SEEDを変えるとほとんど改善されないことがあるかもしれません。)

もちろんケースバイケースなところはあり、SVCのカーネル選択などは精度に大きく影響することもあります。

とことん精度を高めたい場面では、ぜひ最適化に取り組んでいただきたいところですが、そもそもの「機械学習モデル選択」や「データ前処理」などの方が重要度が高いことが多いです。

機械学習モデルの改善作業の優先度は、よく検討してみてください。


以上、Optunaによりハイパーパラメータを最適化する様々な方法をご紹介しました。

ブラックボックス最適化は、機械学習以外にも様々な分野での活用が期待できます。

ぜひ、ご自身でもOptunaについてもっと色々と学んでみてください。

このあとは、グループワークにチャレンジしていただきます。